In [1]:
# 기초 전처리
import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm import notebook
from math import log as ln
%matplotlib inline

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

# 집객시설 데이터 로드

In [2]:
geo = pd.read_csv('./상권_집객시설/서울시 우리마을가게 상권분석서비스(상권-집객시설).csv', encoding='cp949')
# 0값이 nan값으로 되어 있기에 0으로 변환
geo.fillna(0, inplace=True)
geo.head(2)

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,집객시설_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,초등학교_수,중학교_수,고등학교_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수
0,2021,2,D,발달상권,1001175,서울 강동구 고덕역,65,1.0,5.0,0.0,0.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
1,2021,2,D,발달상권,1001177,서울 중구 약수역_2,24,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
geo =geo[geo['상권_구분_코드_명'] == '골목상권']
# 2020년과 2021년 데이터만 추출
geo = geo[(geo['기준_년_코드']==2021)]

# 2020년 3분기, 2020년 4분기, 2021년 1분기, 2021년 2분기 데이터만 추출하기 위한 조건 생성
기준년_2021_2분기= ((geo['기준_년_코드']==2021) & (geo['기준_분기_코드']==2))

# 초등/ 중/ 고등학교를 하나로 묶고 대학교는 따로 함(10대의 소비층이 매우 적기 때문)
geo['초/중/고등학교_수'] = geo['초등학교_수'] + geo['중학교_수'] + geo['고등학교_수']
geo.drop(['초등학교_수','중학교_수','고등학교_수'], axis=1, inplace=True)

# 전체 테이블에 boolean 인덱싱
geo = geo[ 기준년_2021_2분기 ]
geo.head(2)

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,집객시설_수,관공서_수,은행_수,종합병원_수,일반_병원_수,약국_수,유치원_수,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수,초/중/고등학교_수
11,2021,2,A,골목상권,1000865,사임당로17길,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12,2021,2,A,골목상권,1000869,서초대로15길,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# base 테이블 로드

In [4]:
df_base = pd.read_csv('base_df.csv')
df_base['상권_코드'] = df_base['상권_코드'].astype(str)
df_base.head(3)

,상권_코드
0,1000001
1,1000002
2,1000003


# 상권별 평균 지가
## 골목상권별 행정동코드 테이블

In [5]:
store_code = pd.read_csv('./법정행정동맵핑/서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='cp949')
store_code.head(3)

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN
1,201810,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,NaN
2,201810,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,NaN


In [6]:
# 골목상권만을 추출
store_code = store_code[store_code['상권_구분_코드'] =='A']
# store_code['행정동_코드'] = store_code['행정동_코드'].astype(int)
# 필요없는 컬럼 정리
store_code.drop(['기준_년월_코드','상권_구분_코드','상권_구분_코드_명', '시군구_코드', '형태정보'], axis=1, inplace=True)
# shape check
print(store_code.shape)
# merge를 위한 변환
store_code['상권_코드'] = store_code['상권_코드'].astype(str)
store_code.head(3)

(1010, 5)


,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,행정동_코드
70,1000334,삼양로93길,201388,459892,11305660
71,1000354,덕릉로60길,203646,459682,11320513
72,1000355,도당로13가길,203280,463008,11320700


In [7]:
store_code = pd.merge(df_base, store_code, on='상권_코드', how='left')
print(store_code.shape)
store_code['행정동_코드'] = store_code['행정동_코드'].astype(str)
store_code.head(3)

(1010, 5)


,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,행정동_코드
0,1000001,계동길,198799,453610,11110600
1,1000002,난계로27길,201996,452630,11110710
2,1000003,돈화문로11가길,198977,452902,11110615


## 서울시별 법정동코드 테이블

In [8]:
area_cat = pd.read_csv('한국행정구역분류_2021.10.1.기준.csv')
area_cat.drop('Unnamed: 4', axis=1, inplace=True)
area_cat.head(3)

,시도,행정구역코드,행정기관코드,법정동코드
0,서울특별시,11.0,1100000000,1100000000
1,서울특별시,11010.0,1111000000,1111000000
2,서울특별시,1101072.0,1111051500,1111010100


In [9]:
# 행정동코드와 법정동코드의 뒷자리 0 두개를 삭제
area_cat = area_cat[area_cat['시도']=='서울특별시']
area_cat['행정기관코드'] = area_cat['행정기관코드'].astype(str)
area_cat['행정기관코드'] = area_cat['행정기관코드'].apply(lambda x: x[:-2])

area_cat['법정동코드'] = area_cat['법정동코드'].astype(str)
area_cat['법정동코드'] = area_cat['법정동코드'].apply(lambda x: x[:-2])
area_cat.head(3)

,시도,행정구역코드,행정기관코드,법정동코드
0,서울특별시,11.0,11000000,11000000
1,서울특별시,11010.0,11110000,11110000
2,서울특별시,1101072.0,11110515,11110101


## 골목상권별 행정코드와 서울시 법정동코드 merge 

In [10]:
df_pre = pd.merge(store_code, area_cat, left_on='행정동_코드', right_on='행정기관코드', how='left')
print(df_pre['상권_코드'].nunique())
df_pre.head(3)

1010


,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,행정동_코드,시도,행정구역코드,행정기관코드,법정동코드
0,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110146
1,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110147
2,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110148


In [11]:
# null값으로 된 법정동코드를 추가
null_list = ['노해로17길', '노해로23길','노해로33길','덕릉로40길','덕릉로41길',
             '도봉로99길', '삼각산로28길', '삼양로77길', '수유로12길', '오패산로67길',
             '한천로109길', '한천로124나길','한천로130길','한천로132길','한천로139가길',
             '한천로140길', '상일로5길', '천호대로219길']
법정동코드 = [11305103, 11305103, 11305103, 11305102, 11305102,
            11305103, 11305103, 11305103, 11305103, 11305102,
            11305102, 11305102, 11305102, 11305102, 11305103,
            11305103, 11740103, 11740103]

In [12]:
for null, num in zip(null_list, 법정동코드):
    null_idx = df_pre[df_pre['상권_코드_명']==null]['법정동코드'].index
    df_pre.iloc[null_idx, 8] = num
    
# 소수화된 법정동코드를 str화
df_pre['법정동코드'] = df_pre['법정동코드'].astype(int)
df_pre['법정동코드'] = df_pre['법정동코드'].astype(str)
df_pre['법정동코드'].isnull().sum()

0

In [13]:
print(df_pre.shape)
df_pre.head(3)

(1732, 9)


,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,행정동_코드,시도,행정구역코드,행정기관코드,법정동코드
0,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110146
1,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110147
2,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110148


## 공시지가 테이블

In [14]:
jiga = pd.read_csv('서울시공시지가(2020).csv', encoding='cp949')
print(jiga.shape)
jiga.head(3)

(909537, 13)


,Unnamed: 0,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자
0,30,1.111010e+18,1111010100,서울특별시 종로구 청운동,1,일반,1,2020,1,4357000,2020-05-29,N,2021-08-03
1,43,1.111010e+18,1111010100,서울특별시 종로구 청운동,1,일반,01월 01일,2020,1,1392000,2020-05-29,N,2021-08-03
2,75,1.111010e+18,1111010100,서울특별시 종로구 청운동,1,일반,01월 02일,2020,1,2520000,2020-05-29,N,2021-08-03


In [15]:
# 공시지가 테이블의 unique 법정동코드와 
# df_pre의 unique 법정동코드를 비교
print(jiga['법정동코드'].nunique(), df_pre['법정동코드'].nunique())

467 416


In [16]:
print(jiga[jiga['기준월']==7]['법정동코드'].nunique())
print(jiga[jiga['기준월']==1]['법정동코드'].nunique())

258
467


In [17]:
# 7월의 법정동코드는 258개, df_pre의 법정동코드는 416
# 1월의 법정동코드는 df_pre의 법정동코드보다 많으므로 1월을 기준으로 잡음
jiga = jiga[jiga['기준월']==1]

# 필요없는 컬럼 삭제
jiga.drop(['Unnamed: 0','고유번호','법정동명','특수지구분코드',
           '특수지구분명','지번', '기준연도','공시일자','표준지여부','데이터기준일자','기준월'], axis=1, inplace=True)
jiga.head(3)

,법정동코드,공시지가
0,1111010100,4357000
1,1111010100,1392000
2,1111010100,2520000


In [18]:
# 법정동코드의 형식을 일치시켜줌
jiga['법정동코드'] = jiga['법정동코드'].apply(lambda x: str(x)[:-2])
jiga.head(3)

,법정동코드,공시지가
0,11110101,4357000
1,11110101,1392000
2,11110101,2520000


## 공시지가 테이블 merge

In [19]:
df_pre = pd.merge(df_pre, jiga, on='법정동코드', how='left')
# 공시지가 null check
print(df_pre['공시지가'].isnull().sum())
df_pre.head(5)

0


,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,행정동_코드,시도,행정구역코드,행정기관코드,법정동코드,공시지가
0,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110146,5352000
1,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110146,5000000
2,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110146,5871000
3,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110146,5871000
4,1000001,계동길,198799,453610,11110600,서울특별시,1101060.0,11110600,11110146,8360000


In [20]:
df_pre.drop(['엑스좌표_값','와이좌표_값','시도','행정구역코드','행정기관코드'],
            axis=1, inplace=True)
# df_pre shape check
print(df_pre.shape)
# 상권코드 숫자 check
print(df_pre['상권_코드'].nunique())
# 후에 loop를 빠르게 돌기 위한 int형 변환
df_pre['법정동코드'] = df_pre['법정동코드'].astype(int)
df_pre.head(5)

(7680604, 5)
1010


,상권_코드,상권_코드_명,행정동_코드,법정동코드,공시지가
0,1000001,계동길,11110600,11110146,5352000
1,1000001,계동길,11110600,11110146,5000000
2,1000001,계동길,11110600,11110146,5871000
3,1000001,계동길,11110600,11110146,5871000
4,1000001,계동길,11110600,11110146,8360000


In [21]:
df_pre = df_pre.groupby(['상권_코드'])['공시지가'].mean().to_frame()
df_pre = df_pre.reset_index()
print(df_pre.shape)
df_pre.head()

(1010, 2)


,상권_코드,공시지가
0,1000001,4.675446e+06
1,1000002,3.689914e+06
2,1000003,1.241185e+07
3,1000004,4.357117e+06
4,1000005,1.612400e+06


# 주변 아파트 현황

In [147]:
apt = pd.read_csv('./상권_아파트/서울시 우리마을가게 상권분석서비스(상권배후지-아파트).csv', encoding='cp949')
apt.head(3)

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,아파트_단지_수,아파트_면적_66_제곱미터_미만_세대_수,아파트_면적_66_제곱미터_세대_수,아파트_면적_99_제곱미터_세대_수,아파트_면적_132_제곱미터_세대_수,아파트_면적_165_제곱미터_세대_수,아파트_가격_1_억_미만_세대_수,아파트_가격_1_억_세대_수,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,아파트_가격_4_억_세대_수,아파트_가격_5_억_세대_수,아파트_가격_6_억_이상_세대_수,아파트_평균_면적,아파트_평균_시가
0,2021,2,A,골목상권,1000507,가로공원로58길,135,1059,322,10,0,0,809,296,286,0,0,0,0,45,96832655
1,2021,2,A,골목상권,1001010,풍성로37가길,423,4401,1604,300,1,20,602,2231,1073,628,1055,581,156,47,180148225
2,2021,2,A,골목상권,1001009,천호옛길,204,2325,1405,316,9,0,197,911,504,667,912,646,218,51,208523475


In [148]:
# 기준 분기 전처리
기준년_2020_3분기= ((apt['기준_년_코드']==2020) & (apt['기준_분기_코드']==3)) 
기준년_2020_4분기= ((apt['기준_년_코드']==2020) & (apt['기준_분기_코드']==4)) 
기준년_2021_1분기= ((apt['기준_년_코드']==2021) & (apt['기준_분기_코드']==1)) 
기준년_2021_2분기= ((apt['기준_년_코드']==2021) & (apt['기준_분기_코드']==2))

apt = apt[기준년_2020_3분기 | 기준년_2020_4분기 | 기준년_2021_1분기 | 기준년_2021_2분기]

print(apt.shape)
print(apt.기준_년_코드.unique())
print(apt.기준_분기_코드.unique())

apt.drop(['기준_년_코드','기준_분기_코드'], axis=1, inplace=True)
apt.head(3)

(4036, 21)
[2021 2020]
[2 1 4 3]


,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,아파트_단지_수,아파트_면적_66_제곱미터_미만_세대_수,아파트_면적_66_제곱미터_세대_수,아파트_면적_99_제곱미터_세대_수,아파트_면적_132_제곱미터_세대_수,아파트_면적_165_제곱미터_세대_수,아파트_가격_1_억_미만_세대_수,아파트_가격_1_억_세대_수,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,아파트_가격_4_억_세대_수,아파트_가격_5_억_세대_수,아파트_가격_6_억_이상_세대_수,아파트_평균_면적,아파트_평균_시가
0,A,골목상권,1000507,가로공원로58길,135,1059,322,10,0,0,809,296,286,0,0,0,0,45,96832655
1,A,골목상권,1001010,풍성로37가길,423,4401,1604,300,1,20,602,2231,1073,628,1055,581,156,47,180148225
2,A,골목상권,1001009,천호옛길,204,2325,1405,316,9,0,197,911,504,667,912,646,218,51,208523475


In [149]:
# 골목상권만 추출
apt = apt[apt['상권_구분_코드_명'] == '골목상권']

# check
print(apt.상권_구분_코드_명.unique())

# 필요없는 컬럼 삭제
apt.drop(['상권_구분_코드','상권_코드_명','상권_구분_코드_명'], axis=1, inplace=True)
print(apt.shape)
apt.head(3)

['골목상권']
(4036, 16)


,상권_코드,아파트_단지_수,아파트_면적_66_제곱미터_미만_세대_수,아파트_면적_66_제곱미터_세대_수,아파트_면적_99_제곱미터_세대_수,아파트_면적_132_제곱미터_세대_수,아파트_면적_165_제곱미터_세대_수,아파트_가격_1_억_미만_세대_수,아파트_가격_1_억_세대_수,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,아파트_가격_4_억_세대_수,아파트_가격_5_억_세대_수,아파트_가격_6_억_이상_세대_수,아파트_평균_면적,아파트_평균_시가
0,1000507,135,1059,322,10,0,0,809,296,286,0,0,0,0,45,96832655
1,1001010,423,4401,1604,300,1,20,602,2231,1073,628,1055,581,156,47,180148225
2,1001009,204,2325,1405,316,9,0,197,911,504,667,912,646,218,51,208523475


In [150]:
# 아파트 평가는 아파트 단지수와
# 아파트 가격 1억 미만 세대수 / 아파트 가격 2~5억 세대수 / 아파트 가격 6억 이상 세대수 
# 아파트 평균 시가가 의미있다고 생각함

# 필요없는 컬럼 삭제
apt.drop(['아파트_면적_66_제곱미터_미만_세대_수', '아파트_면적_66_제곱미터_세대_수',
          '아파트_면적_99_제곱미터_세대_수', '아파트_면적_132_제곱미터_세대_수', 
          '아파트_면적_165_제곱미터_세대_수'], axis=1, inplace=True)
apt.head(3)

,상권_코드,아파트_단지_수,아파트_가격_1_억_미만_세대_수,아파트_가격_1_억_세대_수,아파트_가격_2_억_세대_수,아파트_가격_3_억_세대_수,아파트_가격_4_억_세대_수,아파트_가격_5_억_세대_수,아파트_가격_6_억_이상_세대_수,아파트_평균_면적,아파트_평균_시가
0,1000507,135,809,296,286,0,0,0,0,45,96832655
1,1001010,423,602,2231,1073,628,1055,581,156,47,180148225
2,1001009,204,197,911,504,667,912,646,218,51,208523475


In [151]:
# 아파트 가격이 상권 소득에 영향을 미친다고 가정하여
# 3단계로 나눔
apt['아파트_가격_2억_미만_세대_수'] = apt['아파트_가격_1_억_미만_세대_수'] \
                                 + apt['아파트_가격_1_억_세대_수'] \

apt['아파트_가격_2~4억_세대_수'] = apt['아파트_가격_2_억_세대_수'] \
                              + apt['아파트_가격_3_억_세대_수'] \
                              + apt['아파트_가격_4_억_세대_수']

apt['아파트_가격_5억_이상_세대수'] = apt['아파트_가격_5_억_세대_수'] \
                               + apt['아파트_가격_6_억_이상_세대_수']

# 필요없는 컬럼 제거
apt.drop(['아파트_가격_1_억_미만_세대_수', '아파트_가격_1_억_세대_수',
          '아파트_가격_2_억_세대_수', '아파트_가격_3_억_세대_수',
          '아파트_가격_4_억_세대_수', '아파트_가격_5_억_세대_수',
          '아파트_가격_6_억_이상_세대_수', '아파트_평균_면적'], axis=1, inplace=True)

apt.head(3)

,상권_코드,아파트_단지_수,아파트_평균_시가,아파트_가격_2억_미만_세대_수,아파트_가격_2~4억_세대_수,아파트_가격_5억_이상_세대수
0,1000507,135,96832655,1105,286,0
1,1001010,423,180148225,2833,2756,737
2,1001009,204,208523475,1108,2083,864


In [152]:
# 인덱스별 총 아파트 시가를 계산 
apt['아파트_총_시가'] = apt['아파트_단지_수'] * apt['아파트_평균_시가']

# 새로운 아파트 평균 시가 컬럼을 만들기 위해 삭제
apt.drop('아파트_평균_시가', axis=1, inplace=True)

# 상권코드별 아파트 평균 시가를 위한 작업
총_아파트_수 = apt.groupby(['상권_코드'])['아파트_단지_수'].sum()
총_아파트_시가 = apt.groupby(['상권_코드'])['아파트_총_시가'].sum()

# 아파트 평균 시가 계산 (후에 merge를 하기 위해 테이블화)
아파트_평균_시가 = (총_아파트_수 / 총_아파트_시가).to_frame()
아파트_평균_시가.columns = ['아파트_평균_시가']
아파트_평균_시가

,아파트_평균_시가
상권_코드,
1000001,3.917831e-09
1000002,4.348221e-09
1000003,5.626303e-09
1000004,6.733993e-09
1000005,2.961807e-09
...,...
1001006,4.517399e-09
1001007,6.468466e-09
1001008,5.391740e-09


In [153]:
# 상권코드별 집계
apt = apt.groupby(['상권_코드']).sum()

# 테이블 재구조화
apt = apt.reset_index()

apt.head()

,상권_코드,아파트_단지_수,아파트_가격_2억_미만_세대_수,아파트_가격_2~4억_세대_수,아파트_가격_5억_이상_세대수,아파트_총_시가
0,1000001,544,2800,582,245,138852322968
1,1000002,179,4372,5775,2984,41166258826
2,1000003,32,982,1386,0,5687571448
3,1000004,917,4753,2021,124,136174787035
4,1000005,421,1210,2220,1006,142142959722


In [154]:
# 평균시가 컬럼 추가
apt = pd.merge(apt, 아파트_평균_시가, left_on='상권_코드', right_index=True, how='left')
apt.drop(['아파트_총_시가'], inplace=True, axis=1)
print(apt.shape)
apt.head(3)

(1009, 6)


,상권_코드,아파트_단지_수,아파트_가격_2억_미만_세대_수,아파트_가격_2~4억_세대_수,아파트_가격_5억_이상_세대수,아파트_평균_시가
0,1000001,544,2800,582,245,3.917831e-09
1,1000002,179,4372,5775,2984,4.348221e-09
2,1000003,32,982,1386,0,5.626303e-09


##### 평균 시가에 아파트 단지수가 있으니 이를 빼야하는 거 아닐까?

In [156]:
apt['상권_코드'] = apt['상권_코드'].astype(str)
df_pre = pd.merge(df_pre, apt, on='상권_코드', how='left')

print(df_pre.shape)
df_pre.head(4)

,상권_코드,공시지가,아파트_단지_수,아파트_가격_2억_미만_세대_수,아파트_가격_2~4억_세대_수,아파트_가격_5억_이상_세대수,아파트_평균_시가
0,1000001,4.675446e+06,544.0,2800.0,582.0,245.0,3.917831e-09
1,1000002,3.689914e+06,179.0,4372.0,5775.0,2984.0,4.348221e-09
2,1000003,1.241185e+07,32.0,982.0,1386.0,0.0,5.626303e-09
3,1000004,4.357117e+06,917.0,4753.0,2021.0,124.0,6.733993e-09


# 임대료 - 공실률
- 단위 : 천원/㎡, %

In [164]:
rent = pd.read_csv('./상권_임대료공실률/report.txt', sep='\t')

# 테이블 재구조화
rent.drop('기간',axis=1, inplace=True) # 기간이 하나밖에 존재하지 않음
rent.drop(0, axis=0, inplace=True)   # 해더 같음
rent.columns=['광역상권','하위상권','임대료','공실률','투자수익률','소득수익률','자본수익률']

rent.head(10)

,광역상권,하위상권,임대료,공실률,투자수익률,소득수익률,자본수익률
1,서울,서울,52.1,9.5,2.03,0.72,1.31
2,도심(시내),소계,85.6,15.2,1.63,0.69,0.94
3,도심(시내),광화문,78.4,23,1.71,0.76,0.94
4,도심(시내),남대문,77,12.6,1.91,0.79,1.13
5,도심(시내),동대문,45.1,11.9,2.04,0.68,1.37
6,도심(시내),명동,208,37.3,0.32,0.37,-0.05
7,도심(시내),시청,60.4,8.4,1.87,0.81,1.06
8,도심(시내),을지로,42.7,5.4,1.88,0.7,1.18
9,도심(시내),종로,76.5,9.8,1.83,0.76,1.07
10,도심(시내),충무로,48.9,5.7,1.39,0.7,0.69


In [168]:
a = pd.read_csv('./법정행정동맵핑/pos00006m.csv')
a = a[a['ctpv_nm']=='서울특별시']
a

,lgdng_cd,ctpv_nm,ctgg_nm,adstrd_nm,adstrd_en_nm,lgdng_nm,adstrd_cd,admn_inst_cd
0,1100000000,서울특별시,서울특별시,서울특별시,Seoul,서울특별시,11.0,1100000000
1,1111000000,서울특별시,종로구,종로구,Jongno-gu,종로구,11010.0,1111000000
2,1111010100,서울특별시,종로구,청운효자동,Cheongunhyoja-dong,청운동,1101072.0,1111051500
3,1111010200,서울특별시,종로구,청운효자동,Cheongunhyoja-dong,신교동,1101072.0,1111051500
4,1111010300,서울특별시,종로구,청운효자동,Cheongunhyoja-dong,궁정동,1101072.0,1111051500
...,...,...,...,...,...,...,...,...
760,1174010800,서울특별시,강동구,성내제2동,Seongnae 2(i)-dong,성내동,1125066.0,1174065000
761,1174010800,서울특별시,강동구,성내제3동,Seongnae 3(sam)-dong,성내동,1125067.0,1174066000
762,1174010500,서울특별시,강동구,길동,Gil-dong,길동,1125074.0,1174068500
763,1174010600,서울특별시,강동구,둔촌제1동,Dunchon 1(il)-dong,둔촌동,1125070.0,1174069000


In [166]:
rent.하위상권.unique()

array(['서울', '소계', '광화문', '남대문', '동대문', '명동', '시청', '을지로', '종로', '충무로',
       '강남대로', '교대역', '남부터미널', '논현역', '도산대로', '서래마을', '신사역', '압구정동',
       '양재말죽거리', '양재역', '청담', '테헤란로', '공덕역', '당산역', '동교연남', '망원역', '신촌이대',
       '영등포', '홍대합정', '가락시장', '건대입구', '경희대학교', '구로디지털단지역', '군자', '노량진',
       '독산시흥', '뚝섬', '목동', '미아사거리', '불광역', '사당', '상계역', '서울대입구역',
       '성신여자대학교', '수유', '숙명여자대학교', '신림역', '연신내', '오류동역', '왕십리', '용산역',
       '이태원', '잠실송파', '잠실새내역', '장안동', '천호', '청량리', '혜화동', '화곡'],
      dtype=object)

In [169]:
a.adstrd_nm	.unique()

array(['서울특별시', '종로구', '청운효자동', '사직동', '삼청동', '부암동', '평창동', '무악동', '교남동',
       '가회동', '종로1·2·3·4가동', '종로5·6가동', '이화동', '혜화동', '창신제1동', '창신제2동',
       '창신제3동', '숭인제1동', '숭인제2동', '중구', '소공동', '회현동', '명동', '필동', '장충동',
       '광희동', '을지로동', '신당동', '다산동', '약수동', '청구동', '신당제5동', '동화동', '황학동',
       '중림동', '용산구', '후암동', '용산2가동', '남영동', '청파동', '원효로제1동', '원효로제2동',
       '효창동', '용문동', '한강로동', '이촌제1동', '이촌제2동', '이태원제1동', '이태원제2동', '한남동',
       '서빙고동', '보광동', '성동구', '왕십리제2동', '왕십리도선동', '마장동', '사근동', '행당제1동',
       '행당제2동', '응봉동', '금호1가동', '금호2·3가동', '금호4가동', '옥수동', '성수1가제1동',
       '성수1가제2동', '성수2가제1동', '성수2가제3동', '송정동', '용답동', '광진구', '화양동', '군자동',
       '중곡제1동', '중곡제2동', '중곡제3동', '중곡제4동', '능동', '광장동', '자양제1동', '자양제2동',
       '자양제3동', '자양제4동', '구의제1동', '구의제2동', '구의제3동', '동대문구', '용신동', '제기동',
       '전농제1동', '전농제2동', '답십리제1동', '답십리제2동', '장안제1동', '장안제2동', '청량리동',
       '회기동', '휘경제1동', '휘경제2동', '이문제1동', '이문제2동', '중랑구', '면목제2동', '면목제4동',
       '면목제5동', '면목본동', '면목제7동', '면목제3·8동', '상봉제1동',